# Evaluation Scratch Book
Experiments and stuff to be later removed.

## Imports

In [1]:
import grid2op
from grid2op.utils import ScoreL2RPN2022
from grid2op.Agent import DoNothingAgent
import pandas as pd
from pathlib import Path

## Settings

In [2]:
env_name = "rte_case14_realistic_val10"
difficulty = 2
episodes = 100
save_path = "/home/l2rpn/data/evaluation/do_nothing_rte_case14_realistic"
processes = 14

## Initialization

### Environment

In [27]:
env = grid2op.make(env_name, difficulty=difficulty)

### Agent

In [28]:
agent = DoNothingAgent(env.action_space)

## Score Computation

In [29]:
score = ScoreL2RPN2022(env,nb_scenario=episodes, verbose=1)

In [30]:
agent_score = score.get(agent, nb_process = processes)
agent_score_df = pd.DataFrame(agent_score).transpose()
agent_score_df.columns = ["all_scores", "ts_survived", "total_ts"]

Starts the evaluation of the agent
Start the evaluation of the scores


In [31]:
agent_score_df

,all_scores,ts_survived,total_ts
0,0.0,1097.0,8064.0
1,0.0,3399.0,8064.0
2,0.0,1099.0,8064.0
3,0.0,377.0,8064.0
4,0.0,809.0,8064.0
...,...,...,...
95,0.0,521.0,8064.0
96,0.0,511.0,8064.0
97,0.0,1102.0,8064.0
98,0.0,522.0,8064.0


In [32]:
agent_score_df.to_csv(Path(save_path, "l2rpn_2022_score_" + str(difficulty) + ".csv"))

## Score Evaluation

In [3]:
scores = []
d for d in [0, 1, 2, "competition"]:
    scores.append(pd.load_csv(Path(save_path, "l2rpn_2022_score_" + str(d) + ".csv")))

SyntaxError: invalid syntax (2684874982.py, line 2)